# Assignment 2

2.2

Write a DES program to verify this for n=1, n=2 and n=4. Make sure that your result
has a high and known statistical significance. How does the number of measurements
required to attain this, depend on $\rho$?

In [1]:
import random
import simpy
import numpy as np
import pandas as pd
from IPython.display import display, HTML

In [2]:
# Init variables
RANDOM_SEED = 42
N_helpers = [1, 2, 4]  # Number of machines in the queue
serveTime = 6          # Minutes it takes to help a customer
lambdaIAT = 6          # Create a customer every ~7 minutes
SIM_TIME = 80          # Simulation time in minutes
NRUNS = 100             # Amount of runs
customerCount = 50   # Amount of customers

# Choose your queueing system
# resources = [simpy.PriorityResource]
resources = [simpy.Resource]
# resources = [simpy.Resource, simpy.PriorityResource]

# Create Queue object
class Queue(object):
    def __init__(self, env, N, serveTime, resource):
        self.env = env
        self.machine = resource(env, N)
        self.serveTime = serveTime
        self.customerHelped = 0
        self.helperN = N

    def helped(self, customer, customerServeWait=serveTime):
        yield self.env.timeout(customerServeWait)

# Customer with its own serviceTime
def customer(env, name, cw, id, customerServeWait, resource):
    customerServeWait = random.randint(serveTime - 5, serveTime + 5)
    enterQueue = env.now
    if resource == simpy.PriorityResource:
        with cw.machine.request(priority=customerServeWait) as request:
            request.name = name
            request.time = customerServeWait
            yield request

            customerStat[cw.helperN].append(env.now - enterQueue)
            yield env.process(cw.helped(name, customerServeWait=customerServeWait))

    elif resource == simpy.Resource:
        with cw.machine.request() as request:
            request.name = name
            request.time = customerServeWait
            yield request

            customerStat[cw.helperN].append(env.now - enterQueue)
            yield env.process(cw.helped(name, customerServeWait=customerServeWait))


def setup(env, N, serveTime, lambdaIAT, customerCount, resource):
    queue = Queue(env, N, serveTime, resource)

    # Create more customers while the simulation is running
    while queue.customerHelped < customerCount:
        s = np.random.poisson(lambdaIAT, customerCount)
        yield env.timeout(s[queue.customerHelped])
        queue.customerHelped += 1
        env.process(
            customer(env, 'customer %d' % queue.customerHelped, queue,
                     queue.customerHelped,
                     (customerCount - queue.customerHelped), resource))

for resource in resources:
    random.seed(RANDOM_SEED)
    customerStat = {}
    for j in range(NRUNS):
        for N in N_helpers:
            customerStat.setdefault(N, [])
            customerStat.setdefault(str(N) + "P", [])
            customerStat[str(N) + "P"].append(lambdaIAT / (N * serveTime))
            env = simpy.Environment()
            env.process(
                setup(env, N, serveTime, lambdaIAT, customerCount, resource))
            env.run()
    
    # Format data    
    customerStatDf = pd.DataFrame({"Helpers": N_helpers, 
                                  "$\rho$": [np.mean(customerStat[str(N)+'P']).round(decimals=2) for N in N_helpers],
                                  "Average": [np.mean(customerStat[N]).round(decimals=2) for N in N_helpers], 
                                  "Variance": [np.var(customerStat[N]).round(decimals=2) for N in N_helpers],
                                  "std.dev": [np.std(customerStat[N]).round(decimals=2) for N in N_helpers]}).set_index('Helpers')
    
    # Print output    
    print(f"For {str(resource)[33:-2]}")
    display(HTML(customerStatDf.T.to_html()))
    print()


For Resource


Helpers,1,2,4
$\rho$,1.00,0.50,0.25
Average,14.50,0.15,0.00
Variance,197.93,0.44,0.00
std.dev,14.07,0.66,0.00


## 2.3

Also compare the result to that for an M/M/1 queue with shortest job first scheduling,
where you always give priority to the smallest jobs.

In [3]:
# Init variables
RANDOM_SEED = 42
N_helpers = [1, 2, 4]  # Number of machines in the queue
waitTime = 6          # Minutes it takes to help a customer
lambdaIAT = 6          # Create a customer every ~7 minutes
SIM_TIME = 80          # Simulation time in minutes
NRUNS = 10             # Amount of runs
customerCount = 1000   # Amount of customers

# Choose your queueing system
# resources = [simpy.PriorityResource]
# resources = [simpy.Resource]
resources = [simpy.Resource, simpy.PriorityResource]


class Queue(object):
    def __init__(self, env, N, waitTime, resource):
        self.env = env
        self.machine = resource(env, N)
        self.waitTime = waitTime
        self.customerHelped = 0
        self.helperN = N

    def helped(self, customer, customerWait=waitTime):
        yield self.env.timeout(customerWait)


def customer(env, name, cw, id, customerWait, resource):
    customerWait = random.randint(waitTime - 5, waitTime + 5)
    enterQueue = env.now
    if resource == simpy.PriorityResource:
        with cw.machine.request(priority=customerWait) as request:
            request.name = name
            request.time = customerWait
            yield request

            customerStat[cw.helperN].append(env.now - enterQueue)
            yield env.process(cw.helped(name, customerWait=customerWait))

    elif resource == simpy.Resource:
        with cw.machine.request() as request:
            request.name = name
            request.time = customerWait
            yield request

            customerStat[cw.helperN].append(env.now - enterQueue)
            yield env.process(cw.helped(name, customerWait=customerWait))


def setup(env, N, waitTime, lambdaIAT, customerCount, resource):
    queue = Queue(env, N, waitTime, resource)

    # Create more customers while the simulation is running
    while queue.customerHelped < customerCount:
        s = np.random.poisson(lambdaIAT, customerCount)
        yield env.timeout(s[queue.customerHelped])
        queue.customerHelped += 1
        env.process(
            customer(env, 'customer %d' % queue.customerHelped, queue,
                     queue.customerHelped,
                     (customerCount - queue.customerHelped), resource))

for resource in resources:
    random.seed(RANDOM_SEED)
    customerStat = {}
    for j in range(NRUNS):
        for N in N_helpers:
            #         print(f"******** N={N} Helpers ********")
            customerStat.setdefault(N, [])
            customerStat.setdefault(str(N) + "P", [])
            customerStat[str(N) + "P"].append(lambdaIAT / (N * waitTime))
            env = simpy.Environment()
            env.process(
                setup(env, N, waitTime, lambdaIAT, customerCount, resource))
            env.run()
    customerStatDf = pd.DataFrame({"Helpers": N_helpers, 
                                  "$\rho$": [np.mean(customerStat[str(N)+'P']).round(decimals=2) for N in N_helpers],
                                  "Average": [np.mean(customerStat[N]).round(decimals=2) for N in N_helpers], 
                                  "Variance": [np.var(customerStat[N]).round(decimals=2) for N in N_helpers],
                                  "std.dev": [np.std(customerStat[N]).round(decimals=2) for N in N_helpers]}).set_index('Helpers')
    print(f"For {str(resource)[33:-2]}")
    display(HTML(customerStatDf.T.to_html()))
    print()


For Resource


Helpers,1,2,4
$\rho$,1.00,0.50,0.25
Average,65.87,0.14,0.00
Variance,3159.42,0.42,0.00
std.dev,56.21,0.65,0.00



For PriorityResource


Helpers,1,2,4
$\rho$,1.00,0.50,0.25
Average,48.35,0.17,0.00
Variance,33201.06,0.52,0.00
std.dev,182.21,0.72,0.01


## 2.4

Now experiment with different service rate distributions. On the one hand try the
M/D/1 and M/D/n queues, on the other hand try a long-tail distribution. For the latter
you may e.g. use a distribution where 75% of the jobs have an exponential distribution
with an average service time of 1.0 and the remaining 25% an exponential distribution
with an average service time of 5.0 (an example of a hyperexponential distribution).

In [105]:
# Init variables
RANDOM_SEED = 42
N_helpers = [1, 2, 4]  # Number of machines in the queue
waitTime = 6  # Minutes it takes to help a customer
lambdaIAT = 6  # Create a customer every ~7 minutes
serviceScheme = "Long-Tailed"  # Can be "Long-Tailed", "Poisson", "Deterministic", "Inverse"
serviceSchemes = ["Long-Tailed", "Poisson", "Deterministic", "Inverse"]
ltLow = 1  # Lower bound long-tailed distribution
ltHigh = 5  # Upper bound long-tailed distribution
SIM_TIME = 80  # Simulation time in minutes
NRUNS = 20  # Amount of runs
customerCount = 100  # Amount of customers

# Choose your queueing system
# resources = [simpy.PriorityResource]
# resources = [simpy.Resource]
resources = [simpy.Resource, simpy.PriorityResource]


class Queue(object):
    def __init__(self, env, N, waitTime, resource):
        self.env = env
        self.machine = resource(env, N)
        self.waitTime = waitTime
        self.customerHelped = 0
        self.helperN = N

    def helped(self, customer, customerWait=waitTime):
        yield self.env.timeout(customerWait)


def customer(env, name, cw, id, customerWait, resource, serviceScheme):
    if serviceScheme == "Long-Tailed":
        rCheck = random.random()
        if rCheck <= 0.75:
            customerWait = ltLow
        else:
            customerWait = ltHigh
    elif serviceScheme == "Poisson":
        customerWait = random.randint(waitTime - 5, waitTime + 5)
    elif serviceScheme == "Deterministic":
        customerWait = waitTime
    elif serviceScheme == "Inverse":
        customerWait = customerWait

    enterQueue = env.now
    if resource == simpy.PriorityResource:
        with cw.machine.request(priority=customerWait) as request:
            request.name = name
            request.time = customerWait
            yield request

            customerStat[cw.helperN].append(env.now - enterQueue)
            tmpWait.append(env.now - enterQueue)
            yield env.process(cw.helped(name, customerWait=customerWait))

    elif resource == simpy.Resource:
        with cw.machine.request() as request:
            request.name = name
            request.time = customerWait
            yield request

            customerStat[cw.helperN].append(env.now - enterQueue)
            tmpWait.append(env.now - enterQueue)
            yield env.process(cw.helped(name, customerWait=customerWait))


def setup(env, N, waitTime, lambdaIAT, customerCount, resource, serviceScheme):
    queue = Queue(env, N, waitTime, resource)

    # Create more customers while the simulation is running
    while queue.customerHelped < customerCount:
        s = np.random.poisson(lambdaIAT, customerCount)
        yield env.timeout(s[queue.customerHelped])
        queue.customerHelped += 1
        env.process(
            customer(env, 'customer %d' % queue.customerHelped, queue,
                     queue.customerHelped,
                     (customerCount - queue.customerHelped), resource,
                     serviceScheme))


columns = [
    "Rho", "Average", "Variance", "std_dev", "Resource", "Helpers", "run",
    "serviceScheme"
]
resourceStatsRun = pd.DataFrame(columns=columns)
customerAll = pd.DataFrame(columns=columns)
for serviceScheme in serviceSchemes:
    for resource in resources:
        random.seed(RANDOM_SEED)
        customerStat = {}

        for N in N_helpers:
            for j in range(NRUNS):
                tmpWait = []
                customerStat.setdefault(N, [])
                customerStat.setdefault(str(N) + "P", [])
                customerStat[str(N) + "P"].append(lambdaIAT / (N * waitTime))
                env = simpy.Environment()
                env.process(
                    setup(env, N, waitTime, lambdaIAT, customerCount, resource,
                          serviceScheme))
                env.run()

                resourceStatsRun.loc[serviceScheme + "_" + str(resource)[33:-2]
                                     + "_" + str(j) + "_" + str(N)] = [
                                         lambdaIAT / (N * waitTime),
                                         np.mean(tmpWait),
                                         np.var(tmpWait),
                                         np.std(tmpWait),
                                         str(resource)[33:-2], N, j,
                                         serviceScheme
                                     ]

            customerAll.loc[serviceScheme + "_" + str(resource)[33:-2] + "_" + str(N)] = [
                                lambdaIAT / (N * waitTime),
                                np.mean(tmpWait),
                                np.var(tmpWait),
                                np.std(tmpWait),
                                str(resource)[33:-2], N, None, serviceScheme
                            ]

In [102]:
print(resourceStatsRun.shape)
print(customerAll.shape)

(480, 8)
(24, 8)


In [116]:
grp = resourceStatsRun.groupby(["serviceScheme", "Resource", "Helpers"])
grp.describe()

Average                                 \
                                         count       mean        std      min   
serviceScheme Resource         Helpers                                          
Deterministic PriorityResource 1          20.0    11.1580   8.192099     4.18   
                               2          20.0     0.0325   0.035075     0.00   
                               4          20.0     0.0000   0.000000     0.00   
              Resource         1          20.0    11.9565   8.245986     2.47   
                               2          20.0     0.0310   0.024257     0.00   
                               4          20.0     0.0000   0.000000     0.00   
Inverse       PriorityResource 1          20.0  1483.4560   4.923680  1471.03   
                               2          20.0   664.7130   6.574952   654.12   
                               4          20.0   259.3580   7.009112   245.05   
              Resource         1          20.0  2986.7540  13.643887  2960.21   
                               2          20.0  1334.2595  16.837185  1303.89   
                               4          20.0   511.4745  12.623066   487.18   
Long-Tailed   PriorityResource 1          20.0     0.1420   0.081020     0.03   
                               2          20.0     0.0010   0.003078     0.00   
                               4          20.0     0.0000   0.000000     0.00   
              Resource         1          20.0     0.1550   0.079571     0.02   
                               2          20.0     0.0010   0.004472     0.00   
                               4          20.0     0.0000   0.000000     0.00   
Poisson       PriorityResource 1          20.0    16.0605  10.282634     4.62   
                               2          20.0     0.1350   0.068172     0.05   
                               4          20.0     0.0000   0.000000     0.00   
              Resource         1          20.0    23.4470  14.657546     3.98   
                               2          20.0     0.1520   0.049694     0.01   
                               4          20.0     0.0000   0.000000     0.00   

                                                                        \
                                              25%       50%        75%   
serviceScheme Resource         Helpers                                   
Deterministic PriorityResource 1           5.8925     8.415    12.8775   
                               2           0.0000     0.030     0.0475   
                               4           0.0000     0.000     0.0000   
              Resource         1           6.5175     8.580    17.5525   
                               2           0.0100     0.020     0.0500   
                               4           0.0000     0.000     0.0000   
Inverse       PriorityResource 1        1481.2200  1482.415  1486.3375   
                               2         658.9275   664.830   668.5825   
                               4         255.6300   261.025   263.0850   
              Resource         1        2977.4100  2989.545  2995.5125   
                               2        1325.6875  1334.325  1343.5750   
                               4         500.5625   513.255   520.4175   
Long-Tailed   PriorityResource 1           0.0800     0.125     0.1925   
                               2           0.0000     0.000     0.0000   
                               4           0.0000     0.000     0.0000   
              Resource         1           0.1100     0.160     0.2000   
                               2           0.0000     0.000     0.0000   
                               4           0.0000     0.000     0.0000   
Poisson       PriorityResource 1           9.3875    11.350    22.7600   
                               2           0.0850     0.125     0.1800   
                               4           0.0000     0.000     0.0000   
              Resource         1          12.7500    21.610    28.3600   
 

In [106]:
customerAll

,Rho,Average,Variance,std_dev,Resource,Helpers,run,serviceScheme
Long-Tailed_Resource_1,1.00,0.02,3.960000e-02,0.198997,Resource,1,None,Long-Tailed
Long-Tailed_Resource_2,0.50,0.00,0.000000e+00,0.000000,Resource,2,None,Long-Tailed
Long-Tailed_Resource_4,0.25,0.00,0.000000e+00,0.000000,Resource,4,None,Long-Tailed
Long-Tailed_PriorityResource_1,1.00,0.03,2.910000e-02,0.170587,PriorityResource,1,None,Long-Tailed
Long-Tailed_PriorityResource_2,0.50,0.00,0.000000e+00,0.000000,PriorityResource,2,None,Long-Tailed
Long-Tailed_PriorityResource_4,0.25,0.00,0.000000e+00,0.000000,PriorityResource,4,None,Long-Tailed
Poisson_Resource_1,1.00,25.14,4.996804e+02,22.353532,Resource,1,None,Poisson
Poisson_Resource_2,0.50,0.01,9.900000e-03,0.099499,Resource,2,None,Poisson
Poisson_Resource_4,0.25,0.00,0.000000e+00,0.000000,Resource,4,None,Poisson
Poisson_PriorityResource_1,1.00,16.66,1.602124e+03,40.026546,PriorityResource,1,None,Poisson


In [ ]:
resourceStatsRun